# Pull Comments Associated with Posts

This details how to create:

- combined_comments_untagged.csv

## 0. Import Packages

In [1]:
import pandas as pd
from pmaw import PushshiftAPI
import datetime as dt
import numpy as np
import re

from os import listdir
from os.path import isfile, join

pd.set_option('display.max_colwidth', 50)

api = PushshiftAPI()

## 1. Read in'Posts' Data 

To get postid-to-postname mapping.

In [2]:
posts = pd.read_csv('./combined_data/combined_posts.csv')
len(posts)

3178

In [3]:
postid_mapping = pd.Series(posts['title'].values,index=posts['id']).to_dict()

# Total number of posts is 2771, but I've split this into 5 datasets of 500 posts each
len(postid_mapping)

2771

In [5]:
# To pull subsequent batches
post_ids_to_pull = pd.read_csv('./post_ids_to_scrape/post_id_batch_4.csv')
post_ids_to_pull = post_ids_to_pull['post_id'].to_list()
post_ids_to_pull[0:10]

['qrd11e',
 'l9yjc6',
 'kzl110',
 'm2bjre',
 'o61tpk',
 'nzjcfg',
 'ladtdw',
 'qgmfdn',
 'l6baa1',
 'l7fdrx']

In [6]:
# Use this to check which post you've pull until
post_ids_to_pull.index('lco12n')

196

## 2. Pull Comments Data from API

In [27]:
def get_comments(post_ids_to_pull, post_names, batch_start, batch_end, path = './comments/wsb_comments', limit = 100000, score = '>0'):
    
    # Get subset of posts to pull
    post_batch = post_ids_to_pull[batch_start:batch_end]
    
    for post_id in post_batch: 
        gen = api.search_comments(link_id = post_id, # this is the post id for which we want to pull comments from
                                 filter = ['author','body','id','permalink', 'score', 'subreddit', 'link_id'], # list of fields to return, created is returned by default
                                 score = score,
                                 limit = limit # limit on the number of records returned
                                )
        comments_df = pd.DataFrame(gen)

        if not 'created_utc' in comments_df.columns: #if no comments are found and the df is empty, move on to the next post
            continue

        # Clean up
        comments_df['search_term'] = post_id 
        comments_df['post_name'] = postid_mapping[post_id] 

        comments_df.to_csv('{}_{}.csv'.format(path, post_id), header=True, index=False, columns=list(comments_df.axes[1]))

### IMPORTANT NOTES

<b>1. Define index locations of batch start and end</b>

Note that a batch of 50 posts takes about 1 hour to pull, so you can batch these and pull. Need to monitor because the connection can sometimes be dropped.

<b>2. Create the correct directories OR change the path in the function call </b>

Here, I set up the code to dump the comments into a folder named 'comments'. This is important because we have to union everything later.

In [28]:
# Define Parameters, here I'm doing a batch of 1 starting from index 0 to index 25 (not inclusive)
batch_start = 196
batch_end = 197

# csvs will be created in your specified directory
get_comments(post_ids_to_pull = post_ids_to_pull, post_names = postid_mapping, batch_start = batch_start, batch_end = batch_end)

INFO:pmaw.PushshiftAPIBase:99973 result(s) not found in Pushshift
INFO:pmaw.PushshiftAPIBase:Total:: Success Rate: 100.00% - Requests: 10 - Batches: 1 - Items Remaining: 13
INFO:pmaw.PushshiftAPIBase:Total:: Success Rate: 100.00% - Requests: 19 - Batches: 2 - Items Remaining: 0


## 3. Union All Comment Files

Create a folder called 'comments' to get all the data scraped. Comments here are not cleaned yet.

In [7]:
# get filenames of everything successfully scraped
mypath = './comments/'
all_comments = [f for f in listdir(mypath) if isfile(join(mypath, f))]
all_comments.remove('.DS_Store') # remove this line if the file is not in your path. seems to be a mac thing
all_comments[0:10]

['SUPRATIK_0.csv',
 'ELLYN_3.csv',
 'PINXUAN_1.csv',
 'RACHEL_4.csv',
 'RACHEL_6_initial.csv',
 'ZONGXIAN_2.csv']

In [13]:
li = []

for filename in all_comments:
    df = pd.read_csv('./comments/{}'.format(filename), lineterminator='\n')
    # Only for major files
    # Move file name to the front
    df['filename'] = filename
    file_col = df['filename']
    df.drop(labels=['filename'], axis=1,inplace = True)
    df.insert(0, 'filename', file_col)
    
    df = df.rename(columns=lambda x: re.sub('.*post_name.*','post_name',x))
    li.append(df)

combined_comments = pd.concat(li, axis=0, ignore_index=True)
len(combined_comments)

/var/folders/d7/r_slgx394x7008mdj1mj14v40000gn/T/ipykernel_1258/2922877714.py:4: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./comments/{}'.format(filename), lineterminator='\n')


1063533

In [14]:
# combined_comments = pd.concat([pd.read_csv('./comments/{}'.format(f), lineterminator='\n') for f in all_comments])
# len(combined_comments)

In [15]:
combined_comments.head()

filename                author  \
0  SUPRATIK_0.csv             [deleted]   
1  SUPRATIK_0.csv             okgeezeok   
2  SUPRATIK_0.csv  LegitimateInjury5720   
3  SUPRATIK_0.csv             [deleted]   
4  SUPRATIK_0.csv            jawnlerdoe   

                                              body   created_utc       id  \
0                                        [removed]  1612316869.0  glt6x5e   
1                     https://youtu.be/gmq1ueWGKgY  1612139703.0  gljgye5   
2                                   And to da moon  1611968923.0  glbe5gt   
3                                        [removed]  1611913183.0  gl7to3u   
4  I’m all in at open too. We’re in this together.  1611895308.0  gl71p20   

     link_id                                          permalink score  \
0  t3_l7c2a3  /r/wallstreetbets/comments/l7c2a3/fuck_the_hed...   1.0   
1  t3_l7c2a3  /r/wallstreetbets/comments/l7c2a3/fuck_the_hed...   1.0   
2  t3_l7c2a3  /r/wallstreetbets/comments/l7c2a3/fuck_the_hed...   1.0   
3  t3_l7c2a3  /r/wallstreetbets/comments/l7c2a3/fuck_the_hed...   1.0   
4  t3_l7c2a3  /r/wallstreetbets/comments/l7c2a3/fuck_the_hed...   1.0   

        subreddit search_term  \
0  wallstreetbets      l7c2a3   
1  wallstreetbets      l7c2a3   
2  wallstreetbets      l7c2a3   
3  wallstreetbets      l7c2a3   
4  wallstreetbets      l7c2a3   

                                           post_name  
0  Fuck the hedge funds; diamond hands. AMC to th...  
1  Fuck the hedge funds; diamond hands. AMC to th...  
2  Fuck the hedge funds; diamond hands. AMC to th...  
3  Fuck the hedge funds; diamond hands. AMC to th...  
4  Fuck the hedge funds; diamond hands. AMC to th...

#### Posts Removed

Remove posts with error (very low pull rate, deleted etc)

Rachel: ['lco12n', 'lcpz4o', 'lc5690', 'ltnwo9', 'lcjbk4', 'lbxwhy']

Ellyn: ['lc3da4', 'lcflo1', 'lu4yvk']

ZongXian: ['kyhvog', 'rdjvl9', 'ltxv73', 'lu2rdj', 'lbxr6j', 'lu35io', 'lbsxol', 'lbw8w7', 'lc93xa', 'lkw043', 'lck0te', 'lcm2j5']

In [11]:
# Check posts by filename (should be about 540 posts each)
combined_comments['search_term'].groupby(by = combined_comments['filename']).nunique()

filename
ELLYN_3.csv             515
PINXUAN_1.csv           516
RACHEL_4.csv            525
RACHEL_6_initial.csv     64
SUPRATIK_0.csv          524
ZONGXIAN_2.csv          528
Name: search_term, dtype: int64

In [13]:
combined_comments['search_term'].nunique()

2672

### Export to Unioned Comments to CSV

Done!

In [16]:
# export to csv
batchno = 4 # EXAMPLE ONLY
combined_comments.to_csv('combined_comments.csv', sep = ',', index = False)

## 4. Union Sentiment Labels from Tagged Data

In [15]:
# get filenames of all tagged data files
mypath = './tagged_data/'
all_labels = [f for f in listdir(mypath) if isfile(join(mypath, f))]
all_labels.remove('.DS_Store') # remove this line if the file is not in your path. seems to be a mac thing
all_labels[0:10]

['tagged_comments_batch_0_SUPRATIK.csv',
 'tagged_comments_batch_2_ZONGXIAN.csv',
 'tagged_comments_batch_4_TPX.csv',
 'tagged_comments_batch_3_ELLYN.csv',
 'tagged_comments_batch_1_RACHEL.csv']

In [16]:
# Get names
r = re.compile("(?<=_)[A-Z]+(?=.)")
namelist = []
for i, filename in enumerate(all_labels):
    result = r.search(all_labels[i])
    namelist.append(result.group())

namelist

['SUPRATIK', 'ZONGXIAN', 'TPX', 'ELLYN', 'RACHEL']

In [17]:
li = []

for i, filename in enumerate(all_labels):
    df = pd.read_csv('./tagged_data/{}'.format(filename), lineterminator='\n')
    df.columns= df.columns.str.lower()
    df = df.rename(columns=lambda x: re.sub('.*sentiment.*','sentiment',x))
    df['tagger'] = namelist[i]
    li.append(df)

combined_tags = pd.concat(li, axis=0, ignore_index=True)
print(len(combined_tags))
combined_tags.columns

5500


Index(['unnamed: 0', 'author', 'body', 'created_utc', 'id', 'link_id',
       'permalink', 'score', 'subreddit', 'date', 'search_term', 'post_name',
       'day', 'month', 'word_count', 'sentiment', 'tagger', '\r',
       'unnamed: 16'],
      dtype='object')

In [18]:
combined_tags.head()

unnamed: 0         author  \
0           1      nevabraun   
1           2       Awake_4E   
2           3  Cloud9forreal   
3           6       MacCoy69   
4           7        Menuler   

                                                body  created_utc       id  \
0  Thanks but you’ve lost me at \n\n„If look at A...   1612214426  glngetr   
1  Awesome <sunglasses> ! Why the moon <thinking>...   1612213448  glndurd   
2  If you look at AMCs business page you’ll find ...   1612213289  glndfos   
3  I bought 20 today, i am also a retarded dumb m...   1612213186  glnd58d   
4  Ahh, my fellow Retard. I see the more and more...   1612213163  glnd2xt   

     link_id                                          permalink  score  \
0  t3_ladvc3  /r/wallstreetbets/comments/ladvc3/just_bought_...     21   
1  t3_ladvc3  /r/wallstreetbets/comments/ladvc3/just_bought_...     25   
2  t3_ladvc3  /r/wallstreetbets/comments/ladvc3/just_bought_...     58   
3  t3_ladvc3  /r/wallstreetbets/comments/ladvc3/just_bought_...     13   
4  t3_ladvc3  /r/wallstreetbets/comments/ladvc3/just_bought_...     11   

        subreddit                 date search_term  \
0  wallstreetbets  2021-02-01 21:20:26      ladvc3   
1  wallstreetbets  2021-02-01 21:04:08      ladvc3   
2  wallstreetbets  2021-02-01 21:01:29      ladvc3   
3  wallstreetbets  2021-02-01 20:59:46      ladvc3   
4  wallstreetbets  2021-02-01 20:59:23      ladvc3   

                                           post_name  day  month  word_count  \
0  Just bought 860 of AMC, greetings from Germany...    1      2          13   
1  Just bought 860 of AMC, greetings from Germany...    1      2          16   
2  Just bought 860 of AMC, greetings from Germany...    1      2          41   
3  Just bought 860 of AMC, greetings from Germany...    1      2          19   
4  Just bought 860 of AMC, greetings from Germany...    1      2          27   

   sentiment    tagger   \r  unnamed: 16  
0          0  SUPRATIK  NaN          NaN  
1          1  SUPRATIK  NaN          NaN  
2          1  SUPRATIK  NaN          NaN  
3          1  SUPRATIK  NaN          NaN  
4          0  SUPRATIK  NaN          NaN

In [24]:
combined_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   unnamed: 0   5500 non-null   int64  
 1   author       5500 non-null   object 
 2   body         5500 non-null   object 
 3   created_utc  5500 non-null   int64  
 4   id           5500 non-null   object 
 5   link_id      5500 non-null   object 
 6   permalink    5500 non-null   object 
 7   score        5500 non-null   int64  
 8   subreddit    5500 non-null   object 
 9   date         5500 non-null   object 
 10  search_term  5500 non-null   object 
 11  post_name    5500 non-null   object 
 12  day          5500 non-null   int64  
 13  month        5500 non-null   int64  
 14  word_count   5500 non-null   int64  
 15  sentiment    5500 non-null   int64  
 16  tagger       5500 non-null   object 
            2200 non-null   object 
 18  unnamed: 16  0 non-null      float64
dtypes: float64(1

In [26]:
combined_tags[~combined_tags['\r'].isna()][0:10]

unnamed: 0             author  \
1100         331         KingKam217   
1101         332           ubunturd   
1102         333  Im_Blind_And_Deaf   
1103         334        MaybeEgg555   
1104         335          Roman2839   
1105         336          Andrew8th   
1106         338       extracheesed   
1107         339         KingKam217   
1108         342        MaybeEgg555   
1109         343        Fenrir-1919   

                                                   body  created_utc       id  \
1100  Thanks for this!\n\nRealistic price targets fo...   1623008219  h0tsv4p   
1101  This is where I feel safe to talk about BB wit...   1623008217  h0tsuz3   
1102                            this is a good idea btw   1623008208  h0tsu8u   
1103  Bet. I’m buying some.  I have AMC and GME. I’l...   1623008156  h0tsq7b   
1104                  GME- father\nAMC- mother\nBB- son   1623008101  h0tslwl   
1105  BB 25$+ EOW LET'S GOOO <stockup> <stockup> <st...   1623007996  h0tsdov   
1106                             BB to $25 this week ;)   1623007875  h0ts4dx   
1107  Price targets this week (REALISTIC)\n\nGME: 29...   1623007848  h0ts29l   
1108  WELCOME TO WSB or we’re all we post about is B...   1623007812  h0trzf7   
1109  Apes can't lose if Apes fight together <apes> ...   1623007779  h0trww3   

        link_id                                          permalink  score  \
1100  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     32   
1101  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     51   
1102  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     12   
1103  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     31   
1104  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     44   
1105  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     86   
1106  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...    130   
1107  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...    239   
1108  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     57   
1109  t3_nttgr8  /r/wallstreetbets/comments/nttgr8/bb_amc_gme_d...     25   

           subreddit            date search_term  \
1100  wallstreetbets  6/6/2021 19:36      nttgr8   
1101  wallstreetbets  6/6/2021 19:36      nttgr8   
1102  wallstreetbets  6/6/2021 19:36      nttgr8   
1103  wallstreetbets  6/6/2021 19:35      nttgr8   
1104  wallstreetbets  6/6/2021 19:35      nttgr8   
1105  wallstreetbets  6/6/2021 19:33      nttgr8   
1106  wallstreetbets  6/6/2021 19:31      nttgr8   
1107  wallstreetbets  6/6/2021 19:30      nttgr8   
1108  wallstreetbets  6/6/2021 19:30      nttgr8   
1109  wallstreetbets  6/6/2021 19:29      nttgr8   

                                              post_name  day  month  \
1100  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1101  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1102  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1103  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1104  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1105  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1106  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1107  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1108  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   
1109  BB | AMC | GME - Daily Popular Ticker Thread f...    6      6   

      word_count  sentiment    tagger  \r  unnamed: 16  
1100          25          0  ZONGXIAN  \r          NaN  
1101          18          0  ZONGXIAN  \r          NaN  
1102           6          0  ZONGXIAN  \r          NaN  
1103          14          1  ZONGXIAN  \r          NaN  
1104           6          0  ZONGXIAN  \r          NaN  
1105           9          1  ZONGXIAN  \r          NaN  
1106           6          1  ZONGXIAN  \r          NaN  
1107          19         

In [27]:
# Grab only the columns needed
tag_cols = ['id', 'sentiment', 'search_term', 'tagger']
tags_only = combined_tags[tag_cols]

In [28]:
tags_only.describe().transpose()

count      mean       std  min  25%  50%  75%  max
sentiment  5500.0  0.390727  0.602674 -1.0  0.0  0.0  1.0  1.0

In [29]:
tags_only.head()

id  sentiment search_term    tagger
0  glngetr          0      ladvc3  SUPRATIK
1  glndurd          1      ladvc3  SUPRATIK
2  glndfos          1      ladvc3  SUPRATIK
3  glnd58d          1      ladvc3  SUPRATIK
4  glnd2xt          0      ladvc3  SUPRATIK

In [30]:
# Read in file with initial comments
initial_comments = pd.read_csv('./comments/comments_batch_initial.csv')

In [31]:
initial_comments.head()

author                body created_utc       id    link_id  \
0  mastermuffin123  Happy cake day! 💎👐  1612253841  glpj6xb  t3_ladvc3   
1        [deleted]           [removed]  1612242495  glp3x6a  t3_ladvc3   
2        [deleted]           [removed]  1612236576  glot6pr  t3_ladvc3   
3        [deleted]           [removed]  1612234492  glooziv  t3_ladvc3   
4        [deleted]           [removed]  1612231345  gloih2v  t3_ladvc3   

                                           permalink score       subreddit  \
0  /r/wallstreetbets/comments/ladvc3/just_bought_...     2  wallstreetbets   
1  /r/wallstreetbets/comments/ladvc3/just_bought_...     1  wallstreetbets   
2  /r/wallstreetbets/comments/ladvc3/just_bought_...     1  wallstreetbets   
3  /r/wallstreetbets/comments/ladvc3/just_bought_...     1  wallstreetbets   
4  /r/wallstreetbets/comments/ladvc3/just_bought_...     1  wallstreetbets   

  search_term                                          post_name  \
0      ladvc3  Just bought 860 of AMC, greetings from Germany...   
1      ladvc3  Just bought 860 of AMC, greetings from Germany...   
2      ladvc3  Just bought 860 of AMC, greetings from Germany...   
3      ladvc3  Just bought 860 of AMC, greetings from Germany...   
4      ladvc3  Just bought 860 of AMC, greetings from Germany...   

               filename  
0  comments_batch_6.csv  
1  comments_batch_6.csv  
2  comments_batch_6.csv  
3  comments_batch_6.csv  
4  comments_batch_6.csv

In [32]:
# Join comment details to tags
comments_and_tags = tags_only.merge(initial_comments, on = 'id', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
comments_and_tags.head()

id  sentiment search_term    tagger         author  \
0  glngetr          0      ladvc3  SUPRATIK      nevabraun   
1  glndurd          1      ladvc3  SUPRATIK       Awake_4E   
2  glndfos          1      ladvc3  SUPRATIK  Cloud9forreal   
3  glnd58d          1      ladvc3  SUPRATIK       MacCoy69   
4  glnd2xt          0      ladvc3  SUPRATIK        Menuler   

                                                body created_utc    link_id  \
0  Thanks but you’ve lost me at \n\n„If look at A...  1612214426  t3_ladvc3   
1  Awesome 😎! Why the moon 🤔 Let’s shoot AMC out ...  1612213448  t3_ladvc3   
2  If you look at AMCs business page you’ll find ...  1612213289  t3_ladvc3   
3  I bought 20 today, i am also a retarded dumb m...  1612213186  t3_ladvc3   
4  Ahh, my fellow Retard. I see the more and more...  1612213163  t3_ladvc3   

                                           permalink score       subreddit  \
0  /r/wallstreetbets/comments/ladvc3/just_bought_...    21  wallstreetbets   
1  /r/wallstreetbets/comments/ladvc3/just_bought_...    25  wallstreetbets   
2  /r/wallstreetbets/comments/ladvc3/just_bought_...    58  wallstreetbets   
3  /r/wallstreetbets/comments/ladvc3/just_bought_...    13  wallstreetbets   
4  /r/wallstreetbets/comments/ladvc3/just_bought_...    11  wallstreetbets   

                                           post_name              filename  
0  Just bought 860 of AMC, greetings from Germany...  comments_batch_6.csv  
1  Just bought 860 of AMC, greetings from Germany...  comments_batch_6.csv  
2  Just bought 860 of AMC, greetings from Germany...  comments_batch_6.csv  
3  Just bought 860 of AMC, greetings from Germany...  comments_batch_6.csv  
4  Just bought 860 of AMC, greetings from Germany...  comments_batch_6.csv

In [33]:
# Send to CSV
tags_only.to_csv("tags_only.csv")
comments_and_tags.to_csv("comments_and_tags.csv")

## Appendix: Sanity Check: % Scraped

If the amount is quite low like 50%, you can double check but so far from what I see, scraped_comments_count (from the above scraping code, using function search_comments) tends to be correct while total_comments_post_has which is from another function (search_submissions) is wrong.

To find the post, open any csv go to PERMALINK and paste it behind this : www.reddit.com/

Somehow the URL under post does not always lead to the post itself especially if it has been deleted.

In [47]:
# Count comments in unioned file
comments_scraped = combined_comments['search_term'].value_counts().rename_axis('unique_values').to_frame('counts').reset_index()

# Get comment count from in original file
post_to_comments = posts[['id', 'num_comments', 'url']].drop_duplicates()

# Inner join
comments_check = pd.merge(comments_scraped, post_to_comments,left_on='unique_values', right_on = 'id', how='inner', copy=False)
comments_check.rename(columns={"counts": "scraped_comments_count", "num_comments": "total_comments_post_has"}, inplace = True)

# Add % scraped
comments_check['%_comments_scraped'] = round(comments_check['scraped_comments_count'] / comments_check['total_comments_post_has'], 4)*100

# Send to CSV
comments_check.to_csv("comments_check.csv")
comments_check.head()

unique_values  scraped_comments_count      id  total_comments_post_has  \
0        nvsynu                   31871  nvsynu                    34613   
1        nwjtyu                   16868  nwjtyu                    18373   
2        l8s98l                    9773  l8s98l                    10399   
3        nzjcfg                    7264  nzjcfg                    13980   
4        l6sttu                    4654  l6sttu                     4836   

                                                 url  %_comments_scraped  
0  https://www.reddit.com/r/wallstreetbets/commen...               92.08  
1  https://www.reddit.com/r/wallstreetbets/commen...               91.81  
2                https://i.redd.it/tv61zizjkie61.jpg               93.98  
3  https://www.reddit.com/r/wallstreetbets/commen...               51.96  
4                https://i.redd.it/a0to0qe4n1e61.jpg               96.24

In [55]:
posts_scraped = combined_comments.groupby(['filename'])['search_term'].nunique()
posts_scraped

filename
comments_batch_4_dropped.csv    525
comments_batch_6_dropped.csv     64
Name: search_term, dtype: int64